In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import Models.Corn_USA_Yield as cy
import Utilities.GLOBAL as GV
import Utilities.Modeling as um
from datetime import datetime as dt

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [ ]:
scope = cy.Define_Scope()

In [16]:
%%time
raw_data = cy.Get_Data(scope)

Wall time: 7.89 s


In [ ]:
milestones = cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units='in', temp_units='F')

In [ ]:
train_df = cy.Build_Train_DF(raw_data, milestones, intervals, train_DF_instr)
model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [ ]:
instructions=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F')

In [ ]:
%%time
pred_df = cy.Build_Pred_DF(raw_data,milestones,instructions,GV.CUR_YEAR, dt(2022,5,1))
yields = model.predict(pred_df[model.params.index]).values
yields

In [ ]:
pred_df

In [ ]:
milestones['15th_May_pct_planted'].tail()

In [ ]:
milestones.keys()